# Class 5: APIs

## APIs, conceptually

![Diagram showing how online payments work: Expedia talks to Delta, Delta talks to Stripe, Stripe talks to Visa, and Visa talks to Chase](extras/img/apis_conceptually/payments.png)

![Diagram showing how notifications flow through systems](extras/img/apis_conceptually/notifications.png)

![Diagram showing relationship between human languages, programming languages, and APIs](extras/img/apis_conceptually/languages.png)

interactions between systems ↔️

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every call; higher barrier to entry (reading documentation); subject to availability and performance of API

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

_Please pray to the Demo Gods that these all work and there's no profanity_

Pull table from [Wikipedia's list of countries by area](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area#Countries_and_dependencies_by_area):

In [1]:
import pandas as pd

tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area",
    match="Rank",
)
countries = tables[0]
countries

,Rank,Country / Dependency,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Notes
0,NaN,World,"510,072,000 (196,940,000)","148,940,000 (57,510,000)","361,132,000 (139,434,000)",70.8,NaN
1,1,Russia,"17,098,246 (6,601,670)","16,378,410 (6,323,740)","719,836 (277,930)",4.21,"The largest country in the world, which spans ..."
2,NaN,Antarctica,"14,200,000 (5,500,000)","14,200,000 (5,500,000)",0 (0),0,"13,916,000 km2 (5,373,000 sq mi) (98%) of the ..."
3,2,Canada[Note 1],"9,984,670 (3,855,100)","9,093,507 (3,511,023)","891,163 (344,080)",8.93,Largest English- and largest French-speaking c...
4,3/4,China,"9,596,961 (3,705,407)","9,326,410 (3,600,950)","270,550 (104,460)",2.82,"Largest country entirely in Asia, and second-l..."
...,...,...,...,...,...,...,...
260,NaN,Ashmore and Cartier Islands (Australia),5 (1.9),5 (1.9),0 (0),0,NaN
261,NaN,Spratly Islands (disputed),(< 1.9),(< 1.9),0 (0),0,NaN
262,NaN,Coral Sea Islands (Australia),(< 1.2),(< 1.2),0 (0),0,NaN
263,194,Monaco,2.02 (0.78),2.02 (0.78),0 (0),0,European microstate. Smallest country with a c...


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

We only see this because the tables on [fec.gov](https://fec.gov) are [rendered client-side](https://www.solutelabs.com/blog/client-side-vs-server-side-rendering-what-to-choose-when) using their JSON API. That won't be the case for all tables on all sites.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Endpoints are like function names: they represent the information you are retrieving or thing you are trying to do
- Parameters are like function arguments:
   - They allow options to be specified
   - Some are required, some are optional
   - They will differ from one endpoint/function to another
- Anchors won't be used

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the API key with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

In [2]:
import requests

params = {"api_key": "DEMO_KEY", "q": "Jimmy McMillan", "sort": "-first_file_date"}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()
data

{'api_version': '1.0',
 'results': [{'load_date': '2018-02-17T09:16:20+00:00',
   'election_years': [2016],
   'active_through': 2016,
   'candidate_id': 'P60016805',
   'incumbent_challenge_full': 'Open seat',
   'candidate_inactive': False,
   'flags': 'P60016805',
   'office_full': 'President',
   'incumbent_challenge': 'O',
   'district_number': 0,
   'candidate_status': 'N',
   'cycles': [2016, 2018],
   'last_file_date': '2015-10-13',
   'inactive_election_years': None,
   'party': 'REP',
   'has_raised_funds': False,
   'district': '00',
   'office': 'P',
   'last_f2_date': '2015-10-13',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'election_districts': ['00'],
   'party_full': 'REPUBLICAN PARTY',
   'first_file_date': '2015-10-13',
   'federal_funds_flag': False,
   'state': 'US'},
  {'load_date': '2021-12-08T06:50:50+00:00',
   'election_years': [1996, 2012],
   'active_through': 2012,
   'candidate_id': 'P60003290',
   'incumbent_challenge_full': 'Challenger',
   '

### Retrieving nested data

In [3]:
data["results"][0]["name"]

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### [In-class exercise](hw_5.ipynb)

### Reading into a DataFrame

In [4]:
params = {"api_key": "DEMO_KEY", "has_raised_funds": "true"}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()

pd.DataFrame(data["results"])

,load_date,has_raised_funds,last_f2_date,active_through,candidate_id,district_number,inactive_election_years,district,office,election_districts,...,election_years,state,incumbent_challenge_full,flags,candidate_status,office_full,party_full,last_file_date,candidate_inactive,incumbent_challenge
0,2022-08-10T21:11:12+00:00,True,2022-08-10,2022,H2CO07170,7,None,07,H,[07],...,[2022],CO,Open seat,H2CO07170,C,House,REPUBLICAN PARTY,2022-08-10,False,O
1,2022-04-13T21:10:09+00:00,True,2022-03-21,2022,H2UT03280,3,None,03,H,[03],...,[2022],UT,Challenger,H2UT03280,C,House,REPUBLICAN PARTY,2022-03-21,False,C
2,2019-03-27T16:02:41+00:00,True,2018-04-23,2018,S2UT00229,0,None,00,S,"[00, 00]",...,"[2012, 2018]",UT,Open seat,S2UT00229,P,Senate,CONSTITUTION PARTY,2018-04-23,False,O
3,2020-03-18T21:13:37+00:00,True,2019-10-17,2020,H0TX22260,22,None,22,H,[22],...,[2020],TX,Open seat,H0TX22260,C,House,REPUBLICAN PARTY,2019-10-17,False,O
4,2002-03-30T00:00:00+00:00,True,1978-07-05,1978,H6PA16106,16,None,16,H,"[16, 16]",...,"[1976, 1978]",PA,None,H6PA16106,P,House,REPUBLICAN PARTY,1978-07-05,False,None
5,2013-04-26T09:04:30+00:00,True,2012-02-22,2012,H2CA01110,1,None,01,H,[01],...,[2012],CA,Challenger,H2CA01110,P,House,REPUBLICAN PARTY,2012-02-22,False,C
6,2017-08-01T20:57:28+00:00,True,2017-04-26,2018,H8CO06237,6,None,06,H,[06],...,[2018],CO,Challenger,H8CO06237,C,House,DEMOCRATIC PARTY,2017-04-26,False,C
7,2016-11-17T06:10:48+00:00,True,2007-03-13,2008,P80002926,0,None,00,P,[00],...,[2008],US,Open seat,P80002926,N,President,DEMOCRATIC PARTY,2007-03-13,False,O
8,2023-01-12T22:24:01+00:00,True,2022-07-15,2024,H2CA30291,32,None,32,H,"[32, 32]",...,"[2022, 2024]",CA,Challenger,H2CA30291,N,House,DEMOCRATIC PARTY,2022-07-15,False,C
9,2022-07-25T23:03:23+00:00,True,2022-06-06,2022,H2MN07162,7,None,07,H,[07],...,[2022],MN,Challenger,H2MN07162,C,House,DEMOCRATIC-FARMER-LABOR,2022-06-06,False,C


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2023, 1, 24, 3, 40, 35, 61270)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2023, 1, 17, 3, 40, 35, 61270)

In [7]:
start.isoformat()

'2023-01-17T03:40:35.061270'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = "erm2-nwe9"
params = {"$where": f"created_date between '{start.isoformat()}' and '{now.isoformat()}'"}

url = f"https://data.cityofnewyork.us/resource/{data_id}.json"
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '56586305',
  'created_date': '2023-01-22T12:00:00.000',
  'agency': 'DSNY',
  'agency_name': 'Department of Sanitation',
  'complaint_type': 'Derelict Vehicles',
  'descriptor': 'Derelict Vehicles',
  'location_type': 'Street',
  'incident_zip': '11433',
  'incident_address': '155-01 109 AVENUE',
  'street_name': '109 AVENUE',
  'cross_street_1': '155 STREET',
  'cross_street_2': '156 STREET',
  'address_type': 'ADDRESS',
  'city': 'JAMAICA',
  'status': 'Open',
  'resolution_description': 'The Department of Sanitation is in the process of investigating this complaint.',
  'resolution_action_updated_date': '2023-01-22T12:00:00.000',
  'community_board': '12 QUEENS',
  'bbl': '4101440045',
  'borough': 'QUEENS',
  'x_coordinate_state_plane': '1040859',
  'y_coordinate_state_plane': '191654',
  'open_data_channel_type': 'PHONE',
  'park_facility_name': 'Unspecified',
  'park_borough': 'QUEENS',
  'latitude': '40.69254168445434',
  'longitude': '-73.7958650104744',
  'loc

Like the FEC, Socrata uses their own API to populate the tables when browsing data on sites powered by them.

**At-home exercise:** Try filtering a table on the [NYC Open Data Portal](https://data.cityofnewyork.us/), and find the API calls that makes.

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,intersection_street_2,landmark,closed_date,taxi_pick_up_location,facility_type,due_date,bridge_highway_name,bridge_highway_segment,bridge_highway_direction,road_ramp
0,56586305,2023-01-22T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11433,155-01 109 AVENUE,109 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56591089,2023-01-22T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11434,171-11 144 AVENUE,144 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56588330,2023-01-22T02:07:30.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10469,1224 MACE AVENUE,MACE AVENUE,...,THROOP AVENUE,MACE AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56589444,2023-01-22T02:07:23.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10469,1313 HICKS STREET,HICKS STREET,...,FISH AVENUE,HICKS STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,56590626,2023-01-22T02:07:09.000,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Vehicle,Street/Sidewalk,11373,41-86 DENMAN STREET,DENMAN STREET,...,94 STREET,DENMAN STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,56587375,2023-01-21T23:02:32.000,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Vehicle,Street/Sidewalk,10040,BROADWAY,BROADWAY,...,WEST 190 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,56585561,2023-01-21T23:02:27.000,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11233,354 CHAUNCEY STREET,CHAUNCEY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,56584989,2023-01-21T23:02:11.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,10304,225 PARK HILL AVENUE,PARK HILL AVENUE,...,SOBEL COURT,PARK HILL AVENUE,2023-01-22T00:28:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,56590560,2023-01-21T23:01:59.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11370,19 DRIVE,19 DRIVE,...,80 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## Homework 6

In real/ideal world, start with specific question and find data to answer it:

![project flow](extras/img/projectflow.png)

_Source: [Big Data and Social Science](https://textbook.coleridgeinitiative.org/chap-intro.html#the-structure-of-the-book)_

Data needed often doesn't exist or is hard (or impossible) to find/access

![project flow](extras/img/projectflow_amended.png)

[Homework 6](hw_6.ipynb)

## No homework/resubmissions will be accepted after Wednesday 12/21 at 6:45pm ET

In other words, Homework 6 cannot be late.

## Lecture 6

[The Joys (and Woes) of the Craft of Software Engineering](https://cs.calvin.edu/courses/cs/262/kvlinden/references/brooksJoysAndWoes.html)